## Necessary Imports

In [6]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

## Load DB

In [7]:
# Load the persisted 'nomic-embed-text' db
embedding = OllamaEmbeddings(model="nomic-embed-text", show_progress=True)
persist_directory = "./db/db_nomic"

In [ ]:
# # Load the persisted 'all-MiniLM-L6-v2' db
# embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
# persist_directory = "./db/db_minilm"

In [8]:
# Load the persisted ChromaDB vector store

vector_database = Chroma(
    collection_name="local-rag",
    persist_directory=persist_directory,
    embedding_function=embedding
)
print('database loaded')

database loaded


## Select LLM

In [9]:
llm = Ollama(model="llama3.1")
# llm = Ollama(model="mistral")

## Context Retriever

<p style="color:lime;"> Creates 5 different versions of the original question. and then retrieves 3 relevant documents from the database based on those questions.

In [10]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""
    You are an AI language model specialized in physics. Your task is to reformulate the following question into five different versions to retrieve the most relevant physics documents.
    Original question: {question}
    """,
)

retriever = MultiQueryRetriever.from_llm(
    vector_database.as_retriever(search_kwargs={"k": 3}),
    llm,
    prompt = QUERY_PROMPT
)

## Prompt

In [11]:
# RAG prompt
template = """You are a helpful assistant trained to answer physics questions based on the provided context.
Use only the context below to answer the following question as clearly as possible:
{context}
Question: 
{question}
"""

In [12]:
prompt = ChatPromptTemplate.from_template(template)

## RAG Chain

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Query

In [14]:
def post_process_answer(answer):
    # Ensure the answer is focused on the physics topic
    # Optionally trim any irrelevant parts or hallucinated information
    processed_ans =  answer.split("Answer:")[-1].strip()
    return processed_ans if processed_ans else "No relevant info found in the context..."


In [15]:
def getAns(query):
    answer = chain.invoke(query)
    final_ans = post_process_answer(answer)
    output = "query: "+query+"\n"+ "answer:\n"+final_ans
    return output

In [16]:
print(getAns(input()))

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


query: what is linear motion?
answer:
Linear motion, also known as rectilinear motion, refers to the motion of an object in a straight line without any change in direction. It's a type of motion where the object moves from one point to another along a fixed path, with no deviation or turn involved.

In linear motion, the object maintains its speed and direction consistently over time, unless acted upon by an external force that alters its velocity. This can be contrasted with rotational motion, which involves movement around a central axis, and curvilinear motion, which involves movement along a curved path.

Examples of linear motion include:

* A car moving straight down the road
* A ball rolling across a flat surface
* A person walking or running in a straight line

Linear motion is an important concept in physics and engineering, as it forms the basis for understanding more complex motions, such as circular and curvilinear motion.
